In [110]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
sys.path.append('../')
import retrieval_functions as rf

import importlib
importlib.reload(rf)
rf.init_plotting()


In [111]:
experiment="Preprocess"
data=[]
second_data=[]
for framework in ["MOTION","MP-SPDZ","HPMPC"]:
    path= "..\..\..\Ergebnisse\Deviation\\"+framework+"\\"+experiment+"\data\\short_results.csv"
    if framework=="MOTION":
        path2= "..\..\..\Ergebnisse\Deviation\\"+framework+"\\"+experiment+"\data\\full_results.csv"
    frame=pd.read_csv(path ,header=0,sep=';',index_col=False,engine='python')
    frame2=pd.read_csv(path2 ,header=0,sep=';',index_col=False,engine='python')
    second_data.append(frame2)
    frame['framework']=framework
    if framework=="MPyC":
        # assuming your dataframe is df and the time column is 'time'
        frame['runtime_internal(s)'] = pd.to_datetime(frame['runtime_internal(s)'], format='%H:%M:%S.%f').dt.time
        # convert time to seconds
        frame['runtime_internal(s)'] = frame['runtime_internal(s)'].apply(lambda t: t.hour*3600 + t.minute*60 + t.second + t.microsecond*1e-6)
        frame['peakRAM(MiB)']= frame['peakRAM(MiB)']
    elif framework=="HPMPC":
        factor = 1
        if frame['splitroles'].max() == 1:
            factor = 6	
        elif frame['splitroles'].max() == 2:
            factor = 24
        else:
            factor=1
        frame['runtime_internal(s)'] = frame['runtime_chrono(s)']
        frame['runtime_external(s)'] = pd.to_numeric(frame['runtime_external(s)'])* frame['threads'].max()*factor
    data.append(frame)

In [113]:
data[1].columns

Index(['program', 'c.domain', 'adv.model', 'protocol', 'partysize',
       'comp.time(s)', 'comp.peakRAM(MiB)', 'bin.filesize(MiB)',
       'latencies(ms)', 'bandwidths(Mbs)', 'input_size', 'packetdrops(%)',
       'runtime_internal(s)', 'runtime_external(s)', 'peakRAM(MiB)',
       'jobCPU(%)', 'P0commRounds', 'P0dataSent(MB)', 'ALLdataSent(MB)',
       'System time (s)', 'Preprocessing Time(s)', 'Online Phase(s)',
       'framework'],
      dtype='object')

In [114]:
experiment="000 Offline/Online Time Analysis"

In [115]:
## Get the Motion Data
online_runtime_motion=pd.concat([data[0]['latencies(ms)'],data[0]['bandwidths(Mbs)'],data[0]['packetdrops(%)'],second_data[0]['gatesOnline'],second_data[0]['preprocessingTime'],data[0]['runtime_external(s)']],axis=1)
online_runtime_motion['gatesOnline'] = online_runtime_motion['gatesOnline'] / 1000
online_runtime_motion['preprocessingTime'] = online_runtime_motion['preprocessingTime'] / 1000
online_runtime_motion.rename(columns={'gatesOnline': 'OnlineTime(s)', 'preprocessingTime': 'PreprocessTime(s)'}, inplace=True)

In [116]:
#Get the MP-SPDZ Data
online_runtime_mpspdz=pd.concat([data[1].loc[data[1]['protocol']=='replicated-ring']['latencies(ms)'],data[1].loc[data[1]['protocol']=='replicated-ring']['bandwidths(Mbs)'],data[1].loc[data[1]['protocol']=='replicated-ring']['packetdrops(%)'],data[1].loc[data[1]['protocol']=='replicated-ring']['Preprocessing Time(s)'], data[1].loc[data[1]['protocol']=='replicated-ring']['Online Phase(s)'],data[1].loc[data[1]['protocol']=='replicated-ring']['runtime_external(s)']],axis=1)
online_runtime_mpspdz.rename(columns={'Online Phase(s)': 'OnlineTime(s)', 'Preprocessing Time(s)': 'PreprocessTime(s)'}, inplace=True)

In [117]:
#Get MP-SPDZ Data 2
online_runtime_mpspdz2=pd.concat([data[1].loc[data[1]['protocol']=='semi2k']['latencies(ms)'],data[1].loc[data[1]['protocol']=='semi2k']['bandwidths(Mbs)'],data[1].loc[data[1]['protocol']=='semi2k']['packetdrops(%)'],data[1].loc[data[1]['protocol']=='semi2k']['Preprocessing Time(s)'], data[1].loc[data[1]['protocol']=='semi2k']['Online Phase(s)'],data[1].loc[data[1]['protocol']=='semi2k']['runtime_external(s)']],axis=1)
online_runtime_mpspdz2.rename(columns={'Online Phase(s)': 'OnlineTime(s)', 'Preprocessing Time(s)': 'PreprocessTime(s)'}, inplace=True)
online_runtime_mpspdz2

,latencies(ms),bandwidths(Mbs),packetdrops(%),PreprocessTime(s),OnlineTime(s),runtime_external(s)
119,0,25000,0,35.8697,0.389421,36.51
120,0,25000,0,36.1040,0.386333,36.74
121,0,25000,0,35.8059,0.415975,36.48
122,0,25000,0,36.1492,0.399026,36.79
123,0,25000,0,36.0448,0.414978,36.72
...,...,...,...,...,...,...
223,75,25000,0,1282.0200,108.015000,1394.84
224,75,25000,0,1281.5500,107.323000,1393.68
225,75,25000,0,1282.0000,107.909000,1394.73
226,75,25000,0,1282.0800,107.640000,1394.52


In [118]:
data[2].columns

Index(['comp.time(s)', 'comp.peakRAM(MiB)', 'bin.filesize(MiB)', 'ssl',
       'threads', 'datatype', 'function', 'optshare', 'packbool', 'protocol',
       'rxbuffer', 'txbuffer', 'latencies(ms)', 'bandwidths(Mbs)',
       'input_size', 'preprocess', 'splitroles', 'packetdrops(%)',
       'verifybuffer', 'inittime(s)', 'preproc(s)', 'runtime_clock(s)',
       'runtime_getTime(s)', 'runtime_chrono(s)', 'runtime_external(s)',
       'peakRAM(MiB)', 'jobCPU(%)', 'framework', 'runtime_internal(s)'],
      dtype='object')

In [119]:
online_runtime_hpmpc=pd.concat([data[2]['latencies(ms)'],data[2]['bandwidths(Mbs)'],data[2]['packetdrops(%)'],data[2]['preproc(s)'], data[2]['runtime_chrono(s)'],data[2]['runtime_external(s)']],axis=1)
online_runtime_hpmpc.rename(columns={'runtime_chrono(s)': 'OnlineTime(s)', 'preproc(s)': 'PreprocessTime(s)'}, inplace=True)
online_runtime_hpmpc

,latencies(ms),bandwidths(Mbs),packetdrops(%),PreprocessTime(s),OnlineTime(s),runtime_external(s)
0,0,25000,0,0.009343,0.011034,0.04
1,0,25000,0,0.009487,0.010065,0.03
2,0,25000,0,0.009629,0.010021,0.05
3,0,25000,0,0.008695,0.009978,0.03
4,0,25000,0,0.007919,0.009787,0.04
...,...,...,...,...,...,...
114,0,25000,5,0.667276,0.021592,1.11
115,0,25000,5,2.465289,0.227856,5.77
116,0,25000,5,0.012717,0.236324,1.28
117,0,25000,5,0.008661,0.046840,2.28


In [120]:
def get_mean_std_prepro(data, column):
    agg_dict = {
        'PreprocessTime(s)': ['mean', 'std'],
        'OnlineTime(s)': ['mean', 'std'],
        'runtime_external(s)': ['mean', 'std']
    }
    help_dev = data.groupby(column).agg(agg_dict)
    help_dev.columns = ['mean_preproc', 'std_preproc', 'mean_online', 'std_online', 'mean_ex', 'std_ex']
    help_dev[column] = help_dev.index
    return help_dev

In [121]:
data_latency=[online_runtime_hpmpc,online_runtime_mpspdz,online_runtime_mpspdz2,online_runtime_motion]

In [122]:
latency=[]
bandwidth=[]
packetdrop=[]
for data in data_latency:
    latency_filter = (data['bandwidths(Mbs)'] == 25000) & (data['packetdrops(%)']==0)
    bandwidth_filter= (data['latencies(ms)']==0) & (data['packetdrops(%)']==0)
    packetdrop_filter= (data['latencies(ms)']==0) & (data['bandwidths(Mbs)']==25000)
    latency.append(get_mean_std_prepro(data.loc[latency_filter],'latencies(ms)')) 
    bandwidth.append(get_mean_std_prepro(data.loc[bandwidth_filter],'bandwidths(Mbs)'))
    packetdrop.append(get_mean_std_prepro(data.loc[packetdrop_filter],'packetdrops(%)'))
    

In [123]:
label=[]
label.append("HPMPC pas. adv/ hst. maj.")
label.append("MP-SPDZ pas. adv/ hst. maj.")
label.append("MP-SPDZ pas. adv/ dishst. maj.")
label.append("MOTION pas. adv/ dishst. maj.")

In [124]:
strings=[]
for dat in [latency,bandwidth,packetdrop]:
    table_string = "\\begin{table}\n"
    table_string += "\\begin{tabularx}{|c|c|c|c|c|}\n"
    table_string += "\\hline\n"
    table_string += "    & "
    for band in dat[0].index:
        table_string += "    " + str(band) + " & "
    table_string = table_string[:-2] + "\\\\\n"
    table_string += "\\hline\n"
    for s,band in enumerate(dat):
        for typ in ['preproc', 'online', 'ex']:
            if typ=='preproc':
                table_string += "\multirow{6}{2cm}{"+label[s]+"} & Preproc. Time (s)   &  "
            elif typ=='online':
                table_string += " & Online Time (s)   &  "
            else:
                table_string += " & External Runtime (s)  &  "
            
            for i,value in enumerate(band.index):
                table_string += "    " + str(round(band['mean_'+typ].iloc[i],3)) + " "+ u"\u00B1" + " " + str(round(band['std_'+typ].iloc[i],3))  +  " & "
            if typ=='ex':
                table_string = table_string[:-2] + "\\\\\\hline\n"
            else:
                table_string = table_string[:-2] + "\\\\\cline{2-6}\n"

    table_string += "\\end{tabularx}\n"
    table_string += "\\end{table}"
    strings.append(table_string)

In [127]:
print(strings[0])

\begin{table}
\begin{tabularx}{|c|c|c|c|c|}
\hline
    &     0 &     25 &     50 &     75 \\
\hline
\multirow{6}{2cm}{HPMPC pas. adv/ hst. maj.} & Preproc. Time (s)   &      0.009 ± 0.001 &     0.255 ± 0.044 &     0.461 ± 0.071 &     0.655 ± 0.124 \\\cline{2-6}
 & Online Time (s)   &      0.011 ± 0.001 &     0.636 ± 0.045 &     1.234 ± 0.127 &     1.926 ± 0.23 \\\cline{2-6}
 & External Runtime (s)  &      0.034 ± 0.006 &     1.106 ± 0.056 &     2.175 ± 0.125 &     3.264 ± 0.209 \\\hline
\multirow{6}{2cm}{MP-SPDZ pas. adv/ hst. maj.} & Preproc. Time (s)   &      0.014 ± 0.001 &     0.231 ± 0.003 &     0.432 ± 0.0 &     0.617 ± 0.048 \\\cline{2-6}
 & Online Time (s)   &      0.281 ± 0.05 &     17.846 ± 0.019 &     35.268 ± 0.032 &     52.72 ± 0.043 \\\cline{2-6}
 & External Runtime (s)  &      0.468 ± 0.054 &     20.162 ± 0.02 &     39.579 ± 0.036 &     59.024 ± 0.015 \\\hline
\multirow{6}{2cm}{MP-SPDZ pas. adv/ dishst. maj.} & Preproc. Time (s)   &      35.983 ± 0.132 &     450.957 ± 0.